The Original Color Swatch in blib-py
===
I found this color chart image from the web [Bailey Bunch Designs](https://www.baileybunchdesigns.com) and decidded to use the colors here for line colors. This is a short script to select a small area from each color swatch in the image and choose the closest match in the named color table, which is from a blog post at [Webucator]( https://www.webucator.com/blog/2015/03/python-color-constants-module) by Nat Dunn. The published script was modified and adpoted as a sub-module in blib.

In [ ]:
%matplotlib inline

import matplotlib
import numpy as np

import blib
import blib.colors

In [ ]:
# We could map the colors into either the table from cloford or CSS4
useCloford = False
useLab = True

In [ ]:
# The color names to use
if useCloford:
    # Gather the 551 color names into an array of (N x 3) from sub-module colors
    cs = [c[0] for c in blib.colors.items()]
    cn = [[c[1].red, c[1].green, c[1].blue] for c in blib.colors.items()]
    cnp = np.array(cn)
else:
    # Alternatively, we can use CSS4 color names, which is available from matplotlib
    cs = [c[0] for c in matplotlib.colors.CSS4_COLORS.items()]
    cn = [[int(v * 255) for v in matplotlib.colors.hex2color(c[1])] for c in matplotlib.colors.CSS4_COLORS.items()]
    cnp = np.array(cn)

# Lab representation
ln = blib.rgb2lab(cn)
lnp = np.array(ln)

In [ ]:
# Read a color chart image, make a mutable copy
cc = matplotlib.image.imread('blob/color-chart.jpg')
im = np.zeros(cc.shape, dtype=np.uint8)
im[:] = cc[:]

# Dimension on each side to extract color
n = 5
i = np.arange(42)
xx = [int(s) for s in (i % 6) * 120 + 155]
yy = [int(s) for s in np.floor(i / 6) * 120 + 200]

swatch = []
indices = []
for x, y in list(zip(xx, yy)):
    # Compute average of the patch
    patch = im[y:y+n, x:x+n, :]
    clr = [int(s) for s in np.mean(patch, axis=(0, 1))]

    if useLab:
        # Color matching using Lab
        lab = blib.rgb2lab(clr).squeeze()
        rms = np.sqrt(np.mean((lab - lnp) ** 2, axis=1))
    else:
        # Color matching using RGB
        rms = np.sqrt(np.mean((clr - cnp) ** 2, axis=1))

    # Find the best
    idx = np.argmin(rms)
    c = (cs[idx], blib.colors.RGB(cnp[idx, 0], cnp[idx, 1], cnp[idx, 2]))

    # Draw a box around the patch that is used
    im[y-1    , x-1:x+n+1, :] = 0
    im[y+n    , x-1:x+n+1, :] = 0
    im[y-1:y+n+1, x-1    , :] = 0
    im[y-1:y+n+1, x+n    , :] = 0

    # Show the values
    print('i: {:3d} --> [{:3d} {:3d} {:3d}] :{:5.1f} {:16s} = [{:3d} {:3d} {:3d}]'.format(idx, clr[0], clr[1], clr[2], rms[idx], c[0], cnp[idx, 0], cnp[idx, 1], cnp[idx, 2]))
    swatch.append(c)
    indices.append(idx)

Make a Pallete Image
===

This is the part we generate a nice looking image to show off the extrated color

In [ ]:
fig = blib.showSwatch(swatch)

In [ ]:
fig = matplotlib.pyplot.figure(figsize=[x / 36.0 for x in im.shape[1::-1]], dpi=72)
ax = fig.add_axes([0, 0, 1, 1], frameon=False)
ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)
ax.imshow(im)

In [ ]:
# import os
# folder = os.path.expanduser('~/Desktop')
# fig.savefig(folder + '/pallete.png', dpi=3*72)
# matplotlib.pyplot.imsave(folder + '/ex.png', im)